In [ ]:
"""
Get Products List
"""

# Inputs
shpExtent = '/mnt/disk1/jasp/a2autocls2023/ref/lmt_coimbra.shp'
start_time = "20180101" # Use YYYY-MM-DD Format
end_time = "20181231"
out_results = '/mnt/disk1/jasp/a2autocls2023/imgcmb/img_2018p.shp'
platform = "Sentinel-2"
processingL = 'Level-2Ap'

from glass.acq.stl import lst_prod

r = lst_prod(
    shpExtent, start_time, end_time, outshp=out_results,
    platname=platform, plevel=processingL
)

In [ ]:
"""
Get a list with one image for each month of one year
and each cell in grid_shp
"""

from glass.geo.dat.sentinel import lst_prod_by_cell_and_year

grid_shp = '/mnt/c/gwork/gigs_cmcprj/admin/lmt_cmb.shp'
year = 2018
id_col = 'Name'
platform = "Sentinel-2"
processingL = 'Level-2A'
search_shp = '/mnt/c/gwork/gigs_cmcprj/sentinel.shp'
epsg = 3763

lst_prod_by_cell_and_year(
    grid_shp, id_col, year, search_shp,
    platform=platform, processingl=processingL, epsg=epsg
)

In [ ]:
"""
DOWNLOAD Images in Shapefile
"""

shp = '/mnt/d/a2autocls2023/sentinel/img_cmb22_1.shp'
fld = '/mnt/d/a2autocls2023/imgcmb/imgcmb'

from glass.acq.stl import down_imgs

down_imgs(shp, 'uuid', outFolder=fld)

In [ ]:
imgs = '/mnt/disk1/jasp/a2autocls2023/sentinel/img_cmb21.shp'

imgid = 'uuid'

ofolder = '/mnt/disk1/jasp/a2autocls2023/imgcmb/src2021'

In [ ]:
import os
from sentinelsat    import SentinelAPI
from glass.rd.shp   import shp_to_obj
from glass.cons.stl import con_datahub

In [ ]:
# Get global vars
gvar = con_datahub()
user, passw, url = gvar["USER"], gvar["PASSWORD"], gvar["URL"]

# Tbl to df
df_img = shp_to_obj(imgs)

df_img["isd"] = 0
df_img["ist"] = 0

In [ ]:
# API Instance
api = SentinelAPI(user, passw, url)

In [ ]:
def download(row):
    pinfo = api.get_product_odata(row[imgid])
    
    is_on = pinfo["Online"]
    
    if is_on:
        api.download(row[imgid], directory_path=ofolder)
        
        row["isd"] = 1
    
    else:
        if not row.ist:
            api.trigger_offline_retrieval(row[imgid])
            
            row["ist"] = 1
    
    return row

In [ ]:
all_download = 0

while not all_download:
    df_img = df_img.apply(lambda x: download(x), axis=1)
    
    dstatus = df_img.isd.tolist()
    
    if 0 not in dstatus:
        all_download = 1